In [23]:
import pandas as pd
import numpy as np

In [52]:
import psutil
svmem = psutil.virtual_memory()
print (svmem.available) #in bytes 

1859137536


In [2]:
import os
os.path.getsize('./chartevents.csv') 

29184776616

In [5]:
df_sample = pd.read_csv('./chartevents.csv', nrows=10)
df_sample_size = df_sample.memory_usage(index=True).sum()

In [6]:
my_chunk = (2000000000 / df_sample_size)/10
my_chunk = int(my_chunk//1) # we get the integer part
print (my_chunk)

215517


In [ ]:
iter_csv = pd.read_csv(
    'chartevents.csv',
    iterator=True,
    chunksize=my_chunk,
    dtype={'subject_id': int, 'hadm_id': int, 'stay_id': int, 
           'charttime' : str, 'storetime': str, 'itemid': int,
           'value': str, 'valuenum': float, 'valueuom': str, 'warning': int})

In [ ]:
df_result = pd.concat(
    [chunk[(chunk['itemid'] == 220052)|(chunk['itemid'] == 225312)|(chunk['itemid'] == 220181)]
    for chunk in iter_csv])

In [7]:
df_result =  pd.read_csv('blood_pressure.csv')

In [8]:
svents = pd.read_csv('sample_vents.csv')
ids = svents['hadm_id']
df_result = df_result[df_result['hadm_id'].isin(ids)]

In [18]:
#sorted_chartdf.merge(svents, on = 'hadm_id',how = 'inner', rsuffix='_svent')
merged_inner = pd.merge(left=df_result, right=svents, left_on='hadm_id', right_on='hadm_id')

In [20]:
merged_inner['endtime'] = pd.to_datetime(merged_inner['endtime'])
merged_inner['charttime'] = pd.to_datetime(merged_inner['charttime'])
merged_inner['time_diff']= merged_inner['endtime']-merged_inner['charttime']

In [26]:
merged_inner['timediffhour'] = merged_inner['time_diff'] / np.timedelta64(1, 'h') # convert time to hours
#re_vent = vent_diff[vent_diff['timetosecondvent']<48]

In [34]:
merged_inner_pre = merged_inner[merged_inner['timediffhour']>0]

In [42]:
merged_inner_pre['min']=merged_inner_pre.groupby('hadm_id')['timediffhour'].transform('min')

<ipython-input-42-f5089614f74a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_inner_pre['min']=merged_inner_pre.groupby('hadm_id')['timediffhour'].transform('min')


In [72]:
last_value_pre = merged_inner_pre[merged_inner_pre['timediffhour'] == merged_inner_pre['min']] 

In [91]:
x = last_value_pre.groupby('hadm_id')['min'].mean()
meanBP = x.reset_index()

In [92]:
meanBP

,hadm_id,min
0,20000147,0.750000
1,20001305,3.766667
2,20001361,0.316667
3,20001687,0.950000
4,20002267,0.083333
...,...,...
24661,29998113,0.166667
24662,29998115,0.250000
24663,29998399,0.483333
24664,29999098,1.016667


In [86]:
svents

,hadm_id,endtime,value,valueuom
0,25015072,2152-04-10 18:57:00,89.116667,hour
1,28038802,2185-12-20 09:59:00,108.800000,hour
2,21790335,2140-03-11 13:44:00,33.000000,hour
3,24357541,2177-02-10 14:00:00,659.416667,hour
4,23255460,2172-11-20 08:06:00,56.216667,hour
...,...,...,...,...
24674,29960248,2146-12-09 14:23:00,4408.000000,min
24675,29962016,2135-10-30 11:45:00,13085.000000,min
24676,29974575,2131-03-03 19:54:00,5934.000000,min
24677,29987115,2148-02-19 10:00:00,2640.000000,min
